In [1]:
import chess.pgn
import pandas as pd

In [4]:



header_list = ['Event',
 'Site',
 'Date',
 'Round',
 'White',
 'Black',
 'Result',
 'BlackElo',
 'ECO',
 'Opening',
 'Termination',
 'TimeControl',
 'UTCDate',
 'UTCTime',
 'WhiteElo',
]



def open_and_scrape_headers(filepath):
    game_id = 0
    g = pd.DataFrame(columns=['game_id'] + header_list)

    with open(filepath) as f:

        while True:
            game = chess.pgn.read_game(f)
            game_id += 1

            # If there are no more games, exit the loop
            if game is None:
                break

            value_list = [game_id]
            for header in header_list:
                try:
                    value_list.append(game.headers[header])
                except:
                    value_list.append('')

            g.loc[len(g)] = value_list

            if (game_id % 20000 == 0):
                print('Now adding game_id: ' + str(game_id))

    return g




In [10]:
filepath = '/home/amedvedev/PycharmProjects/chessgamesstat/data/drakar1/drakar1-white.pgn'
g = open_and_scrape_headers(filepath).astype({'BlackElo': 'int32', 'WhiteElo': 'int32'})
g

,game_id,Event,Site,Date,Round,White,Black,Result,BlackElo,ECO,Opening,Termination,TimeControl,UTCDate,UTCTime,WhiteElo
0,1,Live Chess,Chess.com,2024.05.02,-,drakar1,tars_case,0-1,2403,B06,,tars_case won on time,180,2024.05.02,02:35:29,2446
1,2,Live Chess,Chess.com,2024.05.02,-,drakar1,michechess89,1-0,2487,D02,,drakar1 won by checkmate,180,2024.05.02,02:46:41,2463
2,3,Live Chess,Chess.com,2024.05.02,-,drakar1,Rodin_Kesh,1-0,2391,A43,,drakar1 won by resignation,180,2024.05.02,02:53:12,2470
3,4,Live Chess,Chess.com,2024.05.04,-,drakar1,Dark_Attack,0-1,2440,D00,,Dark_Attack won by resignation,180,2024.05.04,01:47:18,2452
4,5,Live Chess,Chess.com,2024.05.04,-,drakar1,Mavalazik,0-1,2427,D00,,Mavalazik won on time,180,2024.05.04,05:16:43,2434
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
757,758,Live Chess,Chess.com,2020.01.06,-,drakar1,ratlacodonkg39,1-0,2083,D02,,drakar1 won on time,180,2020.01.06,08:45:31,2136
758,759,Live Chess,Chess.com,2020.01.09,-,drakar1,biciklist,1/2-1/2,2150,A15,,Game drawn by stalemate,180,2020.01.09,06:37:19,2137
759,760,Live Chess,Chess.com,2020.01.09,-,drakar1,chhatturangam,1/2-1/2,2172,A40,,Game drawn by stalemate,180,2020.01.09,07:03:34,2184
760,761,Live Chess,Chess.com,2020.01.09,-,drakar1,G0492,1-0,2168,A11,,drakar1 won by resignation,180,2020.01.09,07:19:24,2200


In [15]:
g[g["BlackElo"] > 2500].sort_values(by=['UTCDate', 'UTCTime'], ascending=True)

,game_id,Event,Site,Date,Round,White,Black,Result,BlackElo,ECO,Opening,Termination,TimeControl,UTCDate,UTCTime,WhiteElo
696,697,Live Chess,Chess.com,2020.08.07,-,drakar1,Aliencowboy,0-1,2548,E10,,Aliencowboy won by resignation,180,2020.08.07,07:10:50,2355
697,698,Live Chess,Chess.com,2020.08.07,-,drakar1,icecreamiscream-inactive,0-1,2519,A43,,icecreamiscream-inactive won by resignation,180,2020.08.07,07:28:48,2338
577,578,Live Chess,Chess.com,2020.09.02,-,drakar1,Sigurbjorn,0-1,2506,B70,,Sigurbjorn won by resignation,180,2020.09.02,17:48:35,2423
578,579,Live Chess,Chess.com,2020.09.02,-,drakar1,Ar4uha,0-1,2545,C01,,Ar4uha won by checkmate,180,2020.09.02,17:56:08,2425
591,592,Live Chess,Chess.com,2020.09.03,-,drakar1,aadiii08,0-1,2503,C50,,aadiii08 won by resignation,180,2020.09.03,07:50:59,2445
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50,51,Live Chess,Chess.com,2024.04.13,-,drakar1,bentsinister,0-1,2509,A41,,bentsinister won by resignation,180,2024.04.13,16:39:41,2462
51,52,Live Chess,Chess.com,2024.04.19,-,drakar1,Themindfulmoves64,1/2-1/2,2531,A13,,Game drawn by timeout vs insufficient material,180,2024.04.19,05:45:50,2473
58,59,Live Chess,Chess.com,2024.04.21,-,drakar1,AvatarRoku7,0-1,2568,A45,,AvatarRoku7 won by resignation,180,2024.04.21,10:42:32,2497
18,19,Live Chess,Chess.com,2024.05.07,-,drakar1,BegusDrasko,1/2-1/2,2516,D00,,Game drawn by repetition,180,2024.05.07,09:07:54,2443
